In [ ]:
import numpy as np
from LAEs.my_functions import *
from LAEs.add_errors import add_errors
from load_single_cats import load_SFG_mock

w_central = central_wavelength()
fwhm_Arr = nb_fwhm(np.arange(60))
w_lya = 1215.67

In [ ]:
sfg_mock_name = 'LAE_12.5deg_z2-4.25_train_jnep_VUDS_deep_0'
mock = load_SFG_mock(sfg_mock_name)

pm_flx = mock.to_numpy().T[:60]
pm_flx, pm_err = add_errors(pm_flx)

i_mag = flux_to_mag(pm_flx[-1], w_central[-1])
r_mag = flux_to_mag(pm_flx[-2], w_central[-2])
g_mag = flux_to_mag(pm_flx[-3], w_central[-3])
for m_Arr in [i_mag, r_mag, g_mag]:
    m_Arr[np.isnan(m_Arr)] = 99.

N_sources = len(i_mag)

In [ ]:
# Continuum estimate
from LAEs.minijpas_LF_and_puricomp import nb_or_3fm_cont
cont_est_flx, cont_est_err = nb_or_3fm_cont(pm_flx, pm_err, 'nb')[:2]

In [ ]:
def dual_selection_NB_candidates(nb, n_sigma=3, ew0_min=30):
    '''
    Function that pre-selects LAE candidates in a dual-mode catalog.
    Inputs:
    nb_flx: J-fluxes for that NB (shape = N)
    nb_err: J-flux errors for that NB (shape = N)
    bb_flx: Fluxes of the BBs (shape = 3xN)
    bb_err: Error flxues of the BBs (shape = 3xN)

    Selection criteria:
    1. n_sigma detection NB
    2. No BB detection (g, r, i)
    3. 30 A EW0
    '''
    nb_flx = pm_flx[nb]
    nb_err = pm_err[nb]
    bb_flx = pm_flx[-2]
    bb_err = pm_err[-2]
    nb_cont_flx = cont_est_flx[nb]
    nb_cont_err = cont_est_err[nb]

    line_flx = nb_flx - nb_cont_flx
    line_err = (nb_err ** 2 + nb_cont_err ** 2) ** 0.5
    n_sigma_detect = line_flx > n_sigma * line_err

    no_bb_detect = (bb_flx / bb_err < 5)#.prod(axis=0).astype(bool)

    EW0_lya = line_flx * fwhm_Arr[nb] / nb_cont_flx
    EW0_lya_cut = EW0_lya > ew0_min

    detection = n_sigma_detect & EW0_lya_cut

    return detection

In [ ]:
nbs_list = np.arange(1, 25)
n_sigma = 4
ew0_min = 50

Lya_cands = np.zeros(N_sources).astype(bool)
nb_Arr = np.ones_like(Lya_cands).astype(int) * -1
for nb_i in nbs_list:
    this_Lya_cands = dual_selection_NB_candidates(nb_i, n_sigma, ew0_min)
    Lya_cands = this_Lya_cands | Lya_cands

    nb_Arr[this_Lya_cands] = nb_i

z_Arr = np.ones(N_sources) * -1
z_Arr[np.where(nb_Arr > 0)] = z_NB(nb_Arr[nb_Arr > 0])

nice_z = np.abs(z_Arr - mock['z']) < 0.16

N_cands = sum(Lya_cands)
N_nice_z = sum(nice_z)
print(N_cands, f'({N_cands / 400:0.2f} deg^-2)')
print(N_nice_z, f'({N_nice_z / 400:0.2f} deg^-2), {N_nice_z / N_cands * 100:0.1f}%')

In [ ]:
L_Arr = EW_L_NB(pm_flx, pm_err, cont_est_flx, cont_est_err, z_Arr, nb_Arr)[2]

In [ ]:
selection = np.where(Lya_cands)[0]

for i, src in enumerate(selection):
    if i == 10:
        break
    
    print(f'i = {i_mag[src]:0.1f}, r = {r_mag[src]:0.1f}, g = {g_mag[src]:0.1f}')
    print(f'L_lya = {mock["L_lya"][src]:0.2f}, EW0 = {mock["EW0"][src]:0.2f}')
    print(f'L_Arr = {L_Arr[src]:0.2f}')
    print(f'z_spec = {mock["z"][src]:0.2f}, z_NB = {z_NB(nb_Arr[src])[0]:0.2f}')

    fig = plt.figure(figsize=(7, 3))

    ax = plot_JPAS_source(pm_flx[:, src], pm_err[:, src])
    ax.axvline(w_central[nb_Arr[src]], ls='--', c='r')
    ax.axvline((mock['z'][src] + 1) * w_lya, ls='--', c='gray')

    plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(5, 4))

ax.scatter(mock['L_lya'][nice_z], L_Arr[nice_z], s=2)
ax.plot([0, 100], [0, 100], ls='--', c='r', zorder=-99)
ax.set_xlim(42.5, 44)
ax.set_ylim(42.5, 44)


plt.show()